# 14.1 Sentence 类第 1 版：单词序列

我们要向这个类的构造方法传入一些文本的字符串，然后可以逐个单词迭代。第一版要实现序列协议，这个类的对象可以迭代，因为所有的序列都可以迭代

In [ ]:
# sentence.py
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self) -> str:
        return 'Sequence(%s)' % reprlib.repr(self.text)

In [ ]:
s = Sentence('"The time has come", the Walrus said')
s

In [ ]:
for word in s.words:
    print(word)

接下来我们还要实现其他 Sentence 类，而且都能通过上面的测试。不过，实例中的实现与其他实现不同，因为这一版的 Sentence 类也是序列，可以按索引获取单词

In [ ]:
s[0],s[1],s[-1]

序列可迭代的原因：iter函数  

解释器需要迭代对象 x 时，会自动调用 iter(x)

内置的 iter 函数有以下作用

1. 检查对象是否实现了 \_\_iter__ 方法，如果实现就调用它，获取一个迭代器
2. 如果没有实现 \_\_iter__ 方法，但是实现了 \_\_getitem__ 方法，Python 会创建一个迭代器，尝试按顺序(从索引 0 开始)获取元素
3. 如果尝试失败，Python 抛出 TypeError 异常，通常会提是 "X object is not iterable"

任何 Python 序列都可迭代的原因是，它们都实现了 \_\_getitem__ 方法。其实，标准的序列也都实现了 \_\_iter__ 方法，因此我们也应这么做。

这也是鸭子类型的极端形式：只要实现 \_\_iter__ 方法或实现 \_\_getitem__ 方法且它的参数是从 0 开始的整数，就认为对象是可迭代的

在白鹅类型中，可迭代对象的定义简单一些，不过没有那么灵活：如果实现了 \_\_iter__ 方法，那么就认为对象是可迭代的。此时，不需要创建子类，也不需要注册，因为 abc.Iterable 类实现了 subclasshook 方法。

# 14.2 可迭代对象和迭代器的对比

可迭代的对象：使用 iter 内置函数可以获取迭代器的对象。如果对象实现了 iter 或 getitem 且 索引从零开始，那也是可迭代的

Python 从可迭代的对象中获取迭代器

下面是一个简单的 for 循环，迭代的一个字符串。这里字符串是可迭代的对象，背后是有迭代器的，只不过我们看不到

In [ ]:
s = 'ABC'
for char in s:
    print(char)

如果没有 for 语句，不得不使用while 循环模拟，要像这样写：

In [ ]:
s = 'ABC'
it = iter(s)    # 使用可迭代对象构建迭代器 it
while True:
    try:
        print(next(it)) # 不断在迭代器上调用 next 函数，获取下一个字符
    except StopIteration:   # 如果没有字符了，迭代器会抛出 StopIteration 异常
        del it  # 释放对 it 的引用，即废弃迭代器对象
        break

StopIteration 异常代表迭代器到头了。Python 语言内部会处理 for 循环和其他迭代上下文（如列表推导、元组拆包 等等）中的 StopIteration 异常。

标准的迭代器接口有两个方法：
* \_\_next__ :返回下一个可用的元素，如果没有元素了则抛出 StopIteration 异常
* \_\_iter__ :返回 self，以便在应该使用可迭代对象的地方使用迭代器，例如在 for 循环中

迭代器的定义：实现了无参数的 \_\_next__ 方法。Python 中的迭代器还实现了 \_\_iter__ 方法，因此迭代器也可以迭代

# 14.3 Sentence 类第 2 版：典型的迭代器

In [ ]:
# 使用迭代器模式实现 Sentence 类
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self) -> str:
        return 'Sequence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return SentenceIterator(self.words)

class SentenceIterator:

    def __init__(self, words):
        self.words = words
        self.index = 0

    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word

    def __iter__(self):
        return self


把 Sentence 变成迭代器：坏主意

构建可迭代对象和迭代器时经常会出现错误，原因是混淆了二者。要知道可迭代的对象有个 \_\_iter__ 方法，每次都实例化一个新的迭代器；而迭代器对象要实现 \_\_next__ 方法，返回单个元素，其次要实现 \_\_iter__ 方法，返回迭代器本身。

因此，迭代器可以迭代，但可迭代的对象不是迭代器。

可迭代的对象一定不能是自身的迭代器，也就是可迭代的对象需要实现 \_\_iter__ 方法，但一定不要实现 \_\_next__ 方法。

# 14.4 Sentence 类第 3 版：生成器函数

In [ ]:
# 使用生成器代替 SentenceIterator 类
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self) -> str:
        return 'Sequence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words:
            yield word  # 产出当前的 word
        return  
        # 这个 return 不是必要的：这个函数可以直接落空，自动返回。
        # 不管有没有 return 语句，生成器函数都不会抛出 StopIteration 异常，而是在生成完全部值之后自动退出。

生成器函数的工作原理

只要 Python 函数的定义体中有 yield 关键字，该函数就是生成器函数。调用生成器函数时，会返回一个生成器对象。也就是说，生成器函数是生成器工厂。

下面以一个特别简单的函数说明生成器的行为：

In [ ]:
def gen_123():
    yield 1
    yield 2
    yield 3

# 只要 Python 函数中包含关键字 yield，该函数就是生成器函数

# gen_123 是函数对象
gen_123

In [ ]:
# 但是调用时，gen_123()返回一个生成器对象
gen_123()

In [ ]:
# 生成器是迭代器，会生成传给 yield 关键字的表达式的值
for i in gen_123():
    print(i)

In [ ]:
# 它的行为和迭代器一致
g = gen_123()
next(g)

In [ ]:
next(g)

In [ ]:
next(g)

In [ ]:
next(g)

生成器函数会创建一个生成器对象，包装生成器函数的定义体。把生成器传给 next(...)函数时，生成器函数会向前，执行函数定义体中的下一个 yield 语句，返回产出的值，并在函数定义体的当前位置暂停。最终，函数的定义体返回时，外层的生成器对象会抛出 StopIteration 异常

下面这个 for 循环清楚的说明了生成器函数定义体的执行过程

In [ ]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

for c in gen_AB():
    print('-->', c)

# 14.5 Sentence 类第 4 版：惰性实现

惰性实现是指尽可能的延后生成指。这样做能节省内存，而却或许还可以避免做无用的处理

目前实现的前几版 Sentence 类都不具有惰性，因为 \_\_init__ 方法急迫地构建好了文本中的单词列表，然后将其绑定在了 self.words 属性上。这样就得处理整个文本，列表使用的内存量可能与文本本身一样多。

re.finditer 函数是 re.findall 函数的惰性版本，返回的不是列表，而是一个生成器，按需生成 re.MatchObject 实例。如果有很多匹配，re.finditer 函数能节省大量内存。

In [ ]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
    
    def __repr__(self) -> str:
        return 'Sequence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()  # match.group() 方法从 MatchObject 实例中提取匹配正则表达式的具体文本

# 14.6 Sentence 类第 5 版：生成器表达式

生成器函数已极大地简化了代码，但使用生成器表达式甚至能把代码变得更短

生成器表达式可以理解为列表推导的惰性版本：不会迫切地构建列表，而是返回一个生成器，按需惰性生成元素。也就是说，如果列表推导是制造列表的工厂，那么生成器表达式就是制造生成器的工厂。

In [ ]:
# 简单地生成器表达式并与列表推导做对比
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

res1 = [x*3 for x in gen_AB()]

In [ ]:
for i in res1:
    print('--->', i)

In [ ]:
res2 = (x*3 for x in gen_AB())
res2    # res2 是一个生成器对象

In [ ]:
for i in res2:
    print('--->', i)

In [ ]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
    
    def __repr__(self) -> str:
        return 'Sequence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

# 14.8 另一个示例：等差数列生成器

本节探讨如何生成不同数字类型的又穷等差数列

In [28]:
# ArithmeticProgression 类：
class ArithmeticProgression:
    def __init__(self, begin, step, end = None):
        self.begin = begin
        self.step = step
        self.end = end  # None -> 无穷数列
    
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)   # 这一行把 self.begin 赋值给 result 
                                                            # 不过先把 self.begin 转换成 begin + step 返回的类型
        forever = self.end is None
        index = 0
        while forever or result < self.end:
            yield result
            index += 1
            result = self.begin + self.step * index # 使用 begin + setp * index 而不是 += 可以降低处理浮点数时积累效应致错的风险

In [29]:
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [31]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

不过如何一个类只是为了构建生成器而去实现 \_\_iter__ 方法，还不如去使用生成器函数

In [ ]:
# aritprog_gen 生成器函数
def aritprog_gen(begin, step, end = None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or  result < end:
        yield result
        index += 1
        result = begin + step * index

使用 itertools 模块生成等差数列

itertools.count 函数返回的生成器能生成多个数。如果不传入参数，itertools.count 函数会生成从零开始的整数数列。  
不过我们可以提供可选的 start 和 step 值

In [32]:
import itertools
gen = itertools.count(1, .5)
next(gen)

1

In [33]:
next(gen)

1.5

In [34]:
next(gen)

2.0

然而 itertools.count 从不停止，因此，如果调用 list(count())， Python会创建一个特别大的列表，超出可用内存。

不过 itertools.takewhile 函数则不同，他会生成一个使用另一个生成器的生成器，在指定条件计算结果为 False 时停止。  
因此可以将他们结合在一起使用

In [35]:
from itertools import count, takewhile
gen = takewhile(lambda n: n <3, count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

In [ ]:
# aritprog_gen 生成器函数的另一种实现、
import itertools

def aritprog_gen(begin, step, end = None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n:n < end, ap_gen)

注意：上面的函数并不是生成器函数，因为它没有 yield 关键字。但它会返回一个生成器，因此它与其他生成器函数一元，是一个生成器工厂函数

# 14.9 标准库中的生成器函数

标准库中提供了很多生成器，比如 os.walk 函数。这个函数在遍历目录树的过程中产出文件名，因此递归搜索文件系统像 for 循环那样简单

不过本节专注于通用的函数：参数为刃影的可迭代对象，返回值是生成器，用于生成选中的、计算出的和重新排列的元素。

## 用于过滤的生成器函数
* itertools.__compress__(it, selector_it) : 并行处理两个可迭代对象，如果 selector_it 中的元素是真值，产出 it 中对应的元素
* itertools.__dropwhile__(predicate, it) : 处理 it，跳过 predicate 的计算结果为真值的元素，然后产出剩下的各个元素(不再做进一步检查)
* __filter__(predicate, it) : 把 it 中的各个元素传给 predicate ，如果 predicate(item) 为真，则产出相应的元素。
* itertools.__filterfalse__(predicate, it) : 与 filter 函数的作用相似，只不过与他相反，是在 predicate 为假值的时候产出对应的元素
* itertools.__islice__(it, stop)/itertools.islice(it, start, stop, step=1) : 产出 it 的切片，作用类似于 s[:stop] 或 s[start:stop:step]，不过 it 可以是任何可迭代的对象，而且这个函数实现的是惰性操作
* itertools.__takewhile__(predicate, it) : predicate 返回真值时产出对应的元素，然后立即停止，不再继续检查

In [38]:
# 示例用法
def vowel(c):
    return c.lower() in 'aeiou'
    
# 把 it 中的各个元素传给 predicate ，如果 predicate(item) 为真，则产出相应的元素。
list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [39]:
import itertools

# 与 filter 函数的作用相似，只不过与他相反，是在 predicate 为假值的时候产出对应的元素
list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [42]:
# 处理 it，跳过 predicate 的计算结果为真值的元素，然后产出剩下的各个元素(不再做进一步检查)
for i in itertools.dropwhile(vowel, 'Aardvark'):
    print(i)

r
d
v
a
r
k


In [43]:
# predicate 返回真值时产出对应的元素，然后立即停止，不再继续检查
list(itertools.takewhile(vowel, 'Aardvark'))

['A', 'a']

In [46]:
# 并行处理两个可迭代对象，如果 selector_it 中的元素是真值，产出 it 中对应的元素
list(itertools.compress('Aardvark', (1, 0, 1, 1, 0, 1)))

['A', 'r', 'd', 'a']

In [48]:
# 产出 it 的切片，作用类似于 s[:stop] 或 s[start:stop:step]，不过 it 可以是任何可迭代的对象，而且这个函数实现的是惰性操作
list(itertools.islice('Aardvark', 4))

['A', 'a', 'r', 'd']

In [49]:
list(itertools.islice('Aardvark', 4, 7))

['v', 'a', 'r']

In [50]:
list(itertools.islice('Aardvark', 1, 7, 2))

['a', 'd', 'a']

## 用于映射的生成器函数

In [51]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
import itertools

In [52]:
# itertools.accumulate(it, [func])
# 产出累积的总和；如果提供了 func，那么把前两个元素传给它，然后将结果和下一个元素给它，以此类推
list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [53]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [54]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [55]:
# enumerate(iterable, start=0)
# 产出由两个元素组成的元组。结果是(index, item )，其中 index 从 start 开始计数，item 则从 iterable 中获取
list(enumerate('albatroz', 1))

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

In [56]:
# map(func, it1, [it2, ..., itN])
# 把 it 中的各个元素传给 func，产出结果；如果传入 N 个可迭代的对象，那么 func 必须能接受 N 个参数，而且要并行处理各个可迭代的对象
import operator
list(map(operator.mul, range(11), range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [57]:
list(map(operator.mul, range(11), [2, 4, 8]))

[0, 4, 16]

In [59]:
# itertools.starmap(func, it)
# 把 it 中的各个元素传给 func，产出结果，输入的可迭代对象应该产出可迭代的元素 iit，然后以 func(*iit) 这种形式调用 func
import itertools
list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

## 合并多个可迭代对象的生成器函数

In [60]:
# itertools.chain(it1, ..., itN)
# 先产出 it1 中的所有元素，然后产出 it2 中的所有元素，以此类推，无缝衔接在一起
list(itertools.chain('ABC', range(3)))

['A', 'B', 'C', 0, 1, 2]

In [61]:
# itertools.chain.from_iterable(it)
# 产出 it 生成的各个可迭代对象中的元素，一个接一个，无缝连接在一起
list(itertools.chain.from_iterable(enumerate('ABC')))

[0, 'A', 1, 'B', 2, 'C']

In [62]:
# itertools.product(it1, ..., itN, repeat=1)
# 计算笛卡尔积；从输入的各个可迭代对象中获取元素，合并成由 N 个元素组成的元组，与嵌套的 for 循环效果一样；repeat 指明重复多少次输入的可迭代对象
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [63]:
list(itertools.product('ABC',repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [65]:
list(itertools.product('ABC'))

[('A',), ('B',), ('C',)]

In [66]:
list(itertools.product(range(3), repeat=2))

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [67]:
list(itertools.product('AB', range(2), repeat=2))

[('A', 0, 'A', 0),
 ('A', 0, 'A', 1),
 ('A', 0, 'B', 0),
 ('A', 0, 'B', 1),
 ('A', 1, 'A', 0),
 ('A', 1, 'A', 1),
 ('A', 1, 'B', 0),
 ('A', 1, 'B', 1),
 ('B', 0, 'A', 0),
 ('B', 0, 'A', 1),
 ('B', 0, 'B', 0),
 ('B', 0, 'B', 1),
 ('B', 1, 'A', 0),
 ('B', 1, 'A', 1),
 ('B', 1, 'B', 0),
 ('B', 1, 'B', 1)]

In [69]:
# zip(it1, ..., itN)
# 并行从输入各个可迭代对象中获取元素，产出由 N 个元素组成的元组，只要有一个可迭代对象的元素到头了，就默默停止
list(zip('ABC', range(3)))

[('A', 0), ('B', 1), ('C', 2)]

In [70]:
list(zip('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [71]:
# itertools.zip_longest(it1, ..., itN, fillvalue = None)
# 并行从输入各个可迭代对象中获取元素，产出由 N 个元素组成的元组，等到最长的可迭代对象到头才停止，空缺的值使用 fillvalue 填充
list(itertools.zip_longest('ABC', range(5), fillvalue= 'TQ'))

[('A', 0), ('B', 1), ('C', 2), ('TQ', 3), ('TQ', 4)]

## 把输入的各个元素扩展成多个输出元素的生成器函数

itertools 模块中的 count 和 repeat 函数返回的生成器 “无中生有” : 这两个函数都不接受可迭代的对象作为输入。  
cycel 生成器会备份输入的可迭代对象，然后重复产出对象中的元素。

In [72]:
# itertools.combinations(it, out_len)
# 把 it 产出的 out_len 个元素组合在一起，然后产出，返回元素的组合，在生成的元组中，元素的顺序无关紧要
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [74]:
# itertools.combinations_with_replacement(it, out_len)
# 把 it 产出的 out_len 个元素组合在一起，然后产出，包含相同元素的组合
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [75]:
# itertools.permutations(it, out_len = None)
# 把 out_len 个 it 产出的元素排列在一起，然后产出这些排列；out_len 的默认值等于 len(list(it))，即返回 out_len 个元素的各种排列
# 在生成的元组中，元素的位置极为重要
list(itertools.permutations('ABC', 2))


[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [79]:
# itertools.repeat(item, [times])
# 不断地产出指定的元素，除非提供 times，指定次数
cc = itertools.repeat('ABC')
list(itertools.islice(cc, 10))

['ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC', 'ABC']

In [80]:
# itertools.count(start = 0, step = 1)
# 从 start 开始不断产出数字，按 setp 指定的步幅增加
ct = itertools.count()
next(ct)
list(itertools.islice(ct, 10))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [82]:
# itertools.cycle(it)
# 从 it 产出各个元素，存储各个元素的副本，然后按顺序重复不断产出各个元素
cy = itertools.cycle('ABC')
list(itertools.islice(cy, 7))

['A', 'B', 'C', 'A', 'B', 'C', 'A']

## 用于重新排列元素的生成器函数

下面会演示 itertools.groupby 函数和内置的reversed 函数的用法。  
注意: groupby 假定输入的可迭代对象要使用分组标准排序；即使不排序，至少也要使用指定的标准分组各个元素

In [83]:
# itertools.groupby(it, key=None)
# 产出由两个元素组成的元素， 形式为(key, group)，其中 key 是分组标准，group 是生成器，用于产出分组里的元素
list(itertools.groupby('LLLAAGGG'))

[('L', <itertools._grouper at 0x2050d219250>),
 ('A', <itertools._grouper at 0x2050d219520>),
 ('G', <itertools._grouper at 0x2050d219b20>)]

In [84]:
for char,group in itertools.groupby('LLLAAGGG'):
    print(char, '->', list(group))

L -> ['L', 'L', 'L']
A -> ['A', 'A']
G -> ['G', 'G', 'G']


In [85]:
# reversed(seq)
# 从后向前，倒序产出 seq 中的元素；seq 必须是序列，或者是实现了 __reversed__ 特殊方法的对象
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear']
animals.sort(key=len)
animals

['rat', 'duck', 'bear', 'eagle', 'giraffe']

In [87]:
list(reversed(animals))

['giraffe', 'eagle', 'bear', 'duck', 'rat']

In [88]:
# itertools.tee(it, n=2)
# 产出一个由 n 个生成器组成的元组，每个生成器用于单独产出输入的可迭代对象中的元素
list(itertools.tee('ABC'))

[<itertools._tee at 0x2050d254300>, <itertools._tee at 0x2050c427a00>]

In [89]:
list(zip(*itertools.tee('ABC')))

[('A', 'A'), ('B', 'B'), ('C', 'C')]

# 14.10 Python 3.3 中新出现的语法: yield from

如果生成器函数需要产出另一个生成器生成的值，传统的解决方案是使用嵌套的 for 循环。

In [90]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i 
s = 'ABC'
t =tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [91]:
# 使用 yield from 实现的 chain
def chain(*iterables):
    for i in iterables:
        yield from i

list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

yield from 完全代替了内层的 for 循环。  
除了代替循环之外，yield from 还会创建通道，把内层生成器直接与外层生成器的客户端联系起来。把生成器当成协程使用时，这个通道特别重要，不仅能为客户的代码生成值，还能使用客户端代码提供的值。

# 14.11 可迭代的归约函数

In [ ]:
# all(it) 
# it 中的所有元素都为真是返回 True，否则返回False

In [ ]:
# any(it)
# it 只要 it 中有元素为真就返回 True，否则返回False

In [ ]:
# max(it, [key=,] [default=]) 
# 返回 it 中最大的元素：key 是排序函数，与 sorted 函数中的一样，如果可迭代的对象为空，返回 default

In [ ]:
# min(it, [key=,] [default=]) 
# 返回 it 中最小的元素：key 是排序函数，与 sorted 函数中的一样，如果可迭代的对象为空，返回 default

In [ ]:
# functools.reduce(func, it, [initial])
# 标签两个元素传给 func，然后把计算结果和第三个元素传给func，以此类推，返回最后的结果，如果提供了 initial，把它当作第一个元素传入

In [ ]:
# sum(it, start=0)
# it 中所有元素的综合，如果提供可选的 start，会把它加上
# 计算浮点数加法时，可以使用 math.fsum 函数提高精度

# 14.12 深入分析 iter 函数

iter 函数有一个鲜为人知的用法：传入两个参数，使用常规的函数或任何可调的对象创建迭代器。这样使用时，第一个参数必须是可调用的对象，用于不断调用(没有参数)，产出各个值，第二个值是哨符，这是个标记值，当可调用的对象返回这个值时，触发迭代器抛出 StopIteration 异常，而不产出哨符。

In [97]:
# 使用 iter 函数掷骰子，直到出 1 为止
import random
def d6():
    return random.randint(1, 6)

d6_iter = iter(d6, 1)
d6_iter

In [98]:
for roll in d6_iter:
    print(roll)

2
4
4


与常规迭代器一样，这个示例中的 d6_iter 对象一旦耗尽了就没用了，如果想要重新开始，必须重新调用 iter(...)，重新构造迭代器。

内置函数 iter 的文档中有个实用的例子。这段代码逐行读取文件，直到遇到空行或达到文件末尾为止：

In [ ]:
with open('mydata.txt') as fp:
    for line in iter(fp.readline, '\n'):
        print(line)